In [1]:
import numpy as np
import matplotlib.pyplot as plt

In [215]:
def singleRing(e, n):
    """Create a single ring of ellipticities
    
    Parameters
    ----------
    e : float
        ellipticity magnitude on the ring
    n : int
        number of uniform samples around ring
    
    Returns
    -------
    ellip : array_like
        Ellipticities as complex numbers.
    """
    return e*np.exp(1j*np.linspace(0,2*np.pi,n,endpoint=False))

def uniformPopulation(emax, n):
    """Create a uniform ellipticity population inside "tophat" region
    
    Parameters
    ----------
    emax : float
        maximum ellipticity magnitude.
    n : int
        number of samples within tophat region of ellipticity space
    
    Returns
    -------
    ellip : array_like
        Ellipticities as complex numbers.
    """
    e = np.random.uniform(0, emax, n)
    phases = np.random.uniform(0, 2*np.pi, n)
    return e * np.exp(1j * phases)

def gaussPopulation(sige, n, maxe=0.8):
    """Create a gaussian ellipticity population inside "tophat" region
    
    Parameters
    ----------
    sige : float
        sigma of centered isotropic 2d Gaussian ellipticity distribution
    n : int
        number of ellipticity samples
    maxe : float
        Hard cut maximum ellipticity magnitude.
    
    Returns
    -------
    ellip : array_like
        Ellipticities as complex numbers.
    """
    e1 = np.random.normal(0.0, sige, n)
    e2 = np.random.normal(0.0, sige, n)
    emag = np.hypot(e1, e2)
    bad = emag > maxe
    while np.any(bad):
        e1[bad] = np.random.normal(0.0, sige, np.sum(bad))
        e2[bad] = np.random.normal(0.0, sige, np.sum(bad))
        emag = np.hypot(e1, e2)
        bad = emag > maxe
    e = e1 + 1j*e2
    return e

In [220]:
def shearRing(e, g):
    """ Apply shear to ellipticities.
    
    Parameters
    ----------
    e : array_like
        Ellipticities as complex numbers
    g : complex
        Shear to apply
        
    Returns
    -------
    eSheared : array_like
        The sheared ellipticities
    """
    return (e+g)/(1+np.conjugate(g)*e)

In [248]:
def resampleStatistic(e, stat, n):
    """
    
    Parameters
    ----------
    e : array_like
        Ellipiticities as complex numbers.
    stat : callable
        The statistic to bootstrap resample.  Takes (resampled) e as argument.
    n : number of bootstrap resamplings to apply
    
    Returns
    -------
    samples : array_like
        The result of applying stat to the resampled ellipticities n times.
    """
    return np.array([stat(np.random.choice(e, size=len(e))) for _ in range(n)])

In [238]:
from scipy.stats import iqr

In [256]:
g = 0.02
nEllip = int(1e5)
nResample = int(1e2)
r = shearRing(singleRing(0.3, nEllip), g)
meanEs = resampleStatistic(r, np.mean, nResample)
medEs = resampleStatistic(r, np.median, nResample)
print("median resampled mean ellipticity: {:10.6f}".format(np.median(meanEs)))
print("IQR of resampled mean ellipticity: {:10.6f}".format(iqr(meanEs)))
print("implied m1-bias: {:10.6f} +/- {:10.6f}".format(
    np.median(meanEs.real)/g-1.0,
    iqr(meanEs.real, scale='normal')/g/np.sqrt(nResample)
))
print()
print("median resampled median ellipticity: {:10.6f}".format(np.median(medEs)))
print("IQR of resampled median ellipticity: {:10.6f}".format(iqr(medEs)))
print("implied m1-bias: {:10.6f} +/- {:10.6f}".format(
    np.median(medEs.real)/g-1.0,
    iqr(medEs.real, scale='normal')/g/np.sqrt(nResample)
))

median resampled mean ellipticity: 0.019938+0.000168j
IQR of resampled mean ellipticity: 0.001006+0.000284j
implied m1-bias:  -0.003098 +/-   0.003728

median resampled median ellipticity: 0.021969+0.000000j
IQR of resampled median ellipticity: 0.001919-0.074945j
implied m1-bias:   0.098439 +/-   0.007114


In [264]:
g = 0.02
nEllip = int(1e7)
nResample = int(1e2)
r = shearRing(uniformPopulation(0.3, nEllip), g)
meanEs = resampleStatistic(r, np.mean, nResample)
medEs = resampleStatistic(r, np.median, nResample)
print("median resampled mean ellipticity: {:10.6f}".format(np.median(meanEs)))
print("IQR of resampled mean ellipticity: {:10.6f}".format(iqr(meanEs)))
print("implied m1-bias: {:10.6f} +/- {:10.6f}".format(
    np.median(meanEs.real)/g-1.0,
    iqr(meanEs.real, scale='normal')/g/np.sqrt(nResample)
))
print()
print("median resampled median ellipticity: {:10.6f}".format(np.median(medEs)))
print("IQR of resampled median ellipticity: {:10.6f}".format(iqr(medEs)))
print("implied m1-bias: {:10.6f} +/- {:10.6f}".format(
    np.median(medEs.real)/g-1.0,
    iqr(medEs.real, scale='normal')/g/np.sqrt(nResample)
))

median resampled mean ellipticity: 0.020012+0.000058j
IQR of resampled mean ellipticity: 0.000055-0.000004j
implied m1-bias:   0.000595 +/-   0.000205

median resampled median ellipticity: 0.020088-0.048153j
IQR of resampled median ellipticity: 0.000024-0.084559j
implied m1-bias:   0.004413 +/-   0.000087


In [265]:
g = 0.02
nEllip = int(1e7)
nResample = int(1e2)
r = shearRing(gaussPopulation(0.3, nEllip), g)
meanEs = resampleStatistic(r, np.mean, nResample)
medEs = resampleStatistic(r, np.median, nResample)
print("median resampled mean ellipticity: {:10.6f}".format(np.median(meanEs)))
print("IQR of resampled mean ellipticity: {:10.6f}".format(iqr(meanEs)))
print("implied m1-bias: {:10.6f} +/- {:10.6f}".format(
    np.median(meanEs.real)/g-1.0,
    iqr(meanEs.real, scale='normal')/g/np.sqrt(nResample)
))
print()
print("median resampled median ellipticity: {:10.6f}".format(np.median(medEs)))
print("IQR of resampled median ellipticity: {:10.6f}".format(iqr(medEs)))
print("implied m1-bias: {:10.6f} +/- {:10.6f}".format(
    np.median(medEs.real)/g-1.0,
    iqr(medEs.real, scale='normal')/g/np.sqrt(nResample)
))

median resampled mean ellipticity: 0.019999-0.000085j
IQR of resampled mean ellipticity: 0.000118-0.000058j
implied m1-bias:  -0.000061 +/-   0.000436

median resampled median ellipticity: 0.021698+0.106635j
IQR of resampled median ellipticity: 0.000130-0.269249j
implied m1-bias:   0.084923 +/-   0.000481
